# Laboratorio 4

## <font color="darkblue">Dataset</font>

<font color="darkblue"><b>Carga del dataset (Movie Reviews) y análisis simple de su contenido

In [8]:
#Archivo con contenido de reviews de películas
g = open('reviews.txt','r') 
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

#Archivo con las etiquetas (POSITIVE,NEGATIVE)
g = open('labels.txt','r') 
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

In [9]:
print len(reviews), len(labels)

25000 25000


In [10]:
print labels[0]
print reviews[0]

POSITIVE
bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   


In [11]:
#Función para imprimir algunos registros por su ID en la lista:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

In [12]:
print("labels.txt \t : \t reviews.txt\n")
pretty_print_review_and_label(2137)
pretty_print_review_and_label(12816)
#pretty_print_review_and_label(6267)
#pretty_print_review_and_label(21934)
#pretty_print_review_and_label(5297)
#pretty_print_review_and_label(4998)

labels.txt 	 : 	 reviews.txt

NEGATIVE	:	this movie is terrible but it has some good effects .  ...
POSITIVE	:	adrian pasdar is excellent is this film . he makes a fascinating woman .  ...


### (P1) Vocabulary and Tokens <font color="darkred">(3 puntos)</font>

In [18]:
#Calcular la cantidad de Tokens total del dataset
# Solución
tokens = []
for review in reviews:
    tokens += review.split()
num_tokens = len(tokens)
num_tokens

6347388

In [19]:
#Calcular el tamaño del vocabulario (tokens únicos) del dataset
# Solución
size_vocabulary = len(set(tokens))
size_vocabulary

74073

#### <hr>
## <font color="darkblue">Feature Extraction

<font color="darkblue"><b>Al finalizar esta sección, debemos contar con 5 vectores de características (X_1, X_2, X_3, X_4, X_5) con la misma cantidad de dimensiones.

In [20]:
from collections import Counter
import numpy as np

### Vector 'y' (labels)

In [21]:
#Se transforma todas las etiquetas a valores numéricos (1,0)
label2index = {}
for i, label in enumerate(list(set([x for x in labels]))):
    label2index[label] = i
    
y = np.array([label2index[x] for x in labels])
y.shape

(25000L,)

### (P2) Matrix X#1: TF-IDF without pre-processing <font color="darkred">(1 punto)</font>

In [24]:
#Obtener la matriz TF-IDF de los reviews
#Sin pre-procesamiento
from sklearn.feature_extraction.text import TfidfVectorizer
# Solución
X_1 = TfidfVectorizer().fit_transform(reviews)

### (P3) Matrix X#2: TF-IDF with pre-processing <font color="darkred">(2 puntos)</font>
Pre-procesamiento: <b>Limpieza de stopwords</b>

In [25]:
# OPCIONAL: remover puntuación
import string
new_reviews = []
for review in reviews:
    new_reviews.append(review.translate(None, string.punctuation))
print len(new_reviews)

25000


In [27]:
#Obtener la matriz TF-IDF de los reviews
from sklearn.feature_extraction.text import TfidfVectorizer
# Solución
X_2 = TfidfVectorizer(stop_words='english').fit_transform(new_reviews)

### Matrix X#3: TF-IDF with Positive/Negative ratios filter
Para este vector de características, se identifican los términos más discriminantes para cada clase de polaridad del review (positivo, negativo). Eso se hace a partir de un análisis de presencia y ausencia entre las clases (enfoque similar al TF-IDF pero no entre documentos, si no entre clases)

In [28]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [29]:
for review, label in zip(reviews, labels):
    if label == 'POSITIVE':
        positive_counts.update(set(review.split(" ")))
    else:
        negative_counts.update(set(review.split(" ")))
total_counts.update(positive_counts)
total_counts.update(negative_counts)

In [30]:
positive_counts.most_common(10)

[('', 12500),
 ('.', 12457),
 ('the', 12381),
 ('and', 12150),
 ('a', 12071),
 ('of', 11857),
 ('to', 11622),
 ('is', 11347),
 ('in', 11103),
 ('this', 11094)]

In [31]:
negative_counts.most_common(10)

[('', 12500),
 ('.', 12465),
 ('the', 12412),
 ('a', 12103),
 ('and', 12014),
 ('of', 11870),
 ('to', 11853),
 ('this', 11545),
 ('it', 11191),
 ('is', 11080)]

In [32]:
frequency_cutoff = 50
pos_neg_ratios = Counter()
for term, cnt in list(total_counts.most_common()):
    if cnt >= frequency_cutoff: # common words only
        pos_neg_ratios[term] = np.log((positive_counts[term]+0.0001) / \
                                      (negative_counts[term]+1))

In [33]:
pos_neg_ratios.most_common(10)

[('excellently', 2.9618324460147543),
 ('delightfully', 2.4638553682474789),
 ('flawless', 2.4336142325930474),
 ('matthau', 2.3513768444638052),
 ('superbly', 2.3025860020845417),
 ('perfection', 2.1731278387649584),
 ('captures', 2.0149035760976659),
 ('wonderfully', 2.0094833153917002),
 ('heartbreaking', 1.9616600685222318),
 ('masterful', 1.9195941740703844)]

In [34]:
list(reversed(pos_neg_ratios.most_common()))[:10]

[('boll', -4.0774374489053864),
 ('uwe', -4.042951272834217),
 ('dreck', -3.2451598004077842),
 ('stinker', -3.2088004893271944),
 ('unwatchable', -3.2088004893271944),
 ('incoherent', -3.0602541281637827),
 ('flimsy', -2.8525980971355276),
 ('unfunny', -2.6810148620698464),
 ('mst', -2.6779616348155488),
 ('yawn', -2.6390323299277534)]

In [35]:
polarity_cutoff = 0.9
review_vocab = [x for x in pos_neg_ratios.keys() \
                if pos_neg_ratios[x] >= polarity_cutoff or \
                   pos_neg_ratios[x] <= -polarity_cutoff]

<b>review_vocab</b> es el nuevo vector de términos filtrado con los más relevantes:

In [36]:
len(review_vocab)

734

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
X_3 = TfidfVectorizer(vocabulary=review_vocab).fit_transform(reviews)
X_3.shape

(25000, 734)

### (P4) Matrix X#4 and X#5: PCA or LSA <font color="darkred">(3 puntos)</font>
Debe aplicar reducción de dimensiones (con PCA o LSA) para las matrices X_1 y X_2
El número de dimensiones que debe obtener debe ser el mismo que el del obtenido para el vector <b>X_3 (review_vocab)</b>

In [40]:
# PCA o LSA sobre X_1
# Solución
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=734, n_iter=5)
X_4 = lsa.fit_transform(X_1)
X_4.shape

(25000L, 734L)

In [43]:
# PCA o LSA sobre X_2
# Solución
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=734, n_iter=5)
X_5 = lsa.fit_transform(X_2)
X_5.shape

(25000L, 734L)

### (P5) Matrix X#1 and X#2: recalculation with max_features <font color="darkred">(1 punto)</font>
Recalcule el vector TF-IDF, usando TfIdfVectorizer, considerando un número maximo de características (<b>max_features</b>) igual a la cantidad de dimensiones obtenido para el vector <b>X_3 (review_vocab)</b>

In [44]:
# Solución
from sklearn.feature_extraction.text import TfidfVectorizer
X_1 = TfidfVectorizer(max_features = 734).fit_transform(reviews)
X_1.shape

(25000, 734)

In [45]:
# Solución
X_2 = TfidfVectorizer(stop_words='english', max_features = 734).fit_transform(reviews)
X_2.shape

(25000, 734)

<hr>
## <font color="darkblue">Model Selection

<font color="darkblue"><b>En esta sección se compararán los resultados de clasificación usando los diferentes vectores</b>

In [46]:
#Ejemplos de algoritmos que pueden probar (pueden usar otros si desean)
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier

<font color="darkblue"><b>Función de ayuda para ejecutar un modelo y que se imprima el resultado de clasificación (accuracy) mediante validación cruzada (k_fold=5):

In [47]:
def run_model(clf, X, y):
    scores = cross_val_score(clf, X, y, cv=5)
    print("%s accuracy: %0.2f (+/- %0.2f)" % \
          (str(clf.__class__).split('.')[-1].replace('>','').replace("'",''), 
          scores.mean(), scores.std() * 2))

In [57]:
def run_models(X, y):
    run_model(LinearSVC(), X, y)
    run_model(SGDClassifier(), X, y)
    run_model(Perceptron(), X, y)
    run_model(PassiveAggressiveClassifier(), X, y)
    run_model(BernoulliNB(), X, y)
#     run_model(MultinomialNB(), X, y)
#     run_model(KNeighborsClassifier(), X, y)
    run_model(NearestCentroid(), X, y)
#     run_model(RandomForestClassifier(n_estimators=50, max_depth=5), X, y)

In [50]:
run_models(X_1, y)

LinearSVC accuracy: 0.85 (+/- 0.01)
SGDClassifier accuracy: 0.84 (+/- 0.01)
Perceptron accuracy: 0.79 (+/- 0.05)
PassiveAggressiveClassifier accuracy: 0.82 (+/- 0.03)
BernoulliNB accuracy: 0.80 (+/- 0.01)
MultinomialNB accuracy: 0.82 (+/- 0.01)
KNeighborsClassifier accuracy: 0.68 (+/- 0.01)
NearestCentroid accuracy: 0.74 (+/- 0.02)
RandomForestClassifier accuracy: 0.80 (+/- 0.00)


### (P6) Split Train-Test <font color="darkred">(2 puntos)

In [51]:
#Dividir el dataset en Train y Test en 0.75-0.25
#Realizar esto para cada vector de características:
# Solución
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X_1, y, test_size=0.25, random_state=1)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_2, y, test_size=0.25, random_state=1)
X3_train, X3_test, y3_train, y3_test = train_test_split(X_3, y, test_size=0.25, random_state=1)
X4_train, X4_test, y4_train, y4_test = train_test_split(X_4, y, test_size=0.25, random_state=1)
X5_train, X5_test, y5_train, y5_test = train_test_split(X_5, y, test_size=0.25, random_state=1)

### (P7) Cross-Validation (kfold=5) <font color="darkred">(3 puntos)
Ejecuten el cross-validation sobre los conjuntos de Train, elija cualquier algoritmo de clasificación (menos el SVM con kernel RBF, que será usado en la siguiente pregunta)

In [58]:
#completar codigo
# Solución
print('X1:')
run_models(X1_train, y1_train)
print('X2:')
run_models(X2_train, y2_train)
print('X3:')
run_models(X3_train, y3_train)
print('X4:')
run_models(X4_train, y4_train)
print('X5:')
run_models(X5_train, y5_train)

X1:
LinearSVC accuracy: 0.85 (+/- 0.02)
SGDClassifier accuracy: 0.84 (+/- 0.02)
Perceptron accuracy: 0.79 (+/- 0.07)
PassiveAggressiveClassifier accuracy: 0.82 (+/- 0.04)
BernoulliNB accuracy: 0.80 (+/- 0.01)
NearestCentroid accuracy: 0.75 (+/- 0.02)
X2:
LinearSVC accuracy: 0.84 (+/- 0.01)
SGDClassifier accuracy: 0.84 (+/- 0.01)
Perceptron accuracy: 0.78 (+/- 0.02)
PassiveAggressiveClassifier accuracy: 0.82 (+/- 0.01)
BernoulliNB accuracy: 0.82 (+/- 0.01)
NearestCentroid accuracy: 0.78 (+/- 0.01)
X3:
LinearSVC accuracy: 0.85 (+/- 0.01)
SGDClassifier accuracy: 0.85 (+/- 0.01)
Perceptron accuracy: 0.79 (+/- 0.02)
PassiveAggressiveClassifier accuracy: 0.83 (+/- 0.01)
BernoulliNB accuracy: 0.85 (+/- 0.01)
NearestCentroid accuracy: 0.81 (+/- 0.00)
X4:
LinearSVC accuracy: 0.88 (+/- 0.01)
SGDClassifier accuracy: 0.87 (+/- 0.03)
Perceptron accuracy: 0.77 (+/- 0.15)
PassiveAggressiveClassifier accuracy: 0.88 (+/- 0.02)
BernoulliNB accuracy: 0.77 (+/- 0.01)
NearestCentroid accuracy: 0.76 (+/- 0.

Identifique los 5 mejores resultados de Vector-Algoritmo, y lístelos (manualmente):

In [ ]:
# Solución
# X4 - LinearSVC accuracy: 0.88 (+/- 0.01)
# X5 - LinearSVC accuracy: 0.88 (+/- 0.01)
# X4 - PassiveAggressiveClassifier accuracy: 0.88 (+/- 0.02)
# X4 - SGDClassifier accuracy: 0.87 (+/- 0.03)
# X5 - SGDClassifier accuracy: 0.87 (+/- 0.01)

### (P8) Test (top5 models) <font color="darkred">(2.5 puntos)

Usando los 5 mejores resultados de Vector-Algoritmo, <b>reentrene su modelo sobre el subconjunto Train total (sin cross-validation) y calcule el accuracy sobre el conjunto de Test</b> respectivo.

In [61]:
#completar codigo
# Solución
model = LinearSVC()
model.fit(X4_train, y4_train)
print('Accuracy: ' + str(model.score(X4_test, y4_test)))

model = LinearSVC()
model.fit(X5_train, y5_train)
print('Accuracy: ' + str(model.score(X5_test, y5_test)))

model = PassiveAggressiveClassifier()
model.fit(X4_train, y4_train)
print('Accuracy: ' + str(model.score(X4_test, y4_test)))

model = SGDClassifier()
model.fit(X4_train, y4_train)
print('Accuracy: ' + str(model.score(X4_test, y4_test)))

model = SGDClassifier()
model.fit(X5_train, y5_train)
print('Accuracy: ' + str(model.score(X5_test, y5_test)))

Accuracy: 0.88272
Accuracy: 0.88096
Accuracy: 0.87904
Accuracy: 0.88208
Accuracy: 0.86752


Identifique el mejor resultado y comente brevemente: ¿Por qué cree que dicho Vector-Algoritmo obtuvo el mejor resultado?

In [62]:
# LinearSVC con el vector X4
# A criterio según resultado

### (P9) SVM and GridSearchCV <font color="darkred">(2.5 puntos)
Optimizar los parámetros <b>(gamma y C)</b> de un algoritmo de clasificación <b>SVM con kernel RBF</b>
<br>http://scikit-learn.org/stable/modules/grid_search.html
<br>http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html#sphx-glr-auto-examples-model-selection-grid-search-digits-py
<br><b> Este procesamiento debe realizarse sobre el Train, y en base a los mejores parámetros obtener el 'accuracy' con el Test</b>

In [63]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
# Solución
parameters = {'kernel':['rbf'], 'C':[1, 10], 'gamma': [0.01, 0.001, 0.0001]}
svr = SVC(random_state = 1)
clf = GridSearchCV(svr, parameters, verbose = 2, cv = 2)
clf.fit(X2_train, y2_train)
clf.score(X2_test, y2_test)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] kernel=rbf, C=1, gamma=0.01 .....................................
[CV] ...................... kernel=rbf, C=1, gamma=0.01, total= 1.5min
[CV] kernel=rbf, C=1, gamma=0.01 .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV] ...................... kernel=rbf, C=1, gamma=0.01, total= 1.6min
[CV] kernel=rbf, C=1, gamma=0.001 ....................................
[CV] ..................... kernel=rbf, C=1, gamma=0.001, total= 1.6min
[CV] kernel=rbf, C=1, gamma=0.001 ....................................
[CV] ..................... kernel=rbf, C=1, gamma=0.001, total= 1.6min
[CV] kernel=rbf, C=1, gamma=0.0001 ...................................
[CV] .................... kernel=rbf, C=1, gamma=0.0001, total= 1.5min
[CV] kernel=rbf, C=1, gamma=0.0001 ...................................
[CV] .................... kernel=rbf, C=1, gamma=0.0001, total= 1.6min
[CV] kernel=rbf, C=10, gamma=0.01 ....................................
[CV] ..................... kernel=rbf, C=10, gamma=0.01, total=  54.5s
[CV] kernel=rbf, C=10, gamma=0.01 ....................................
[CV] ..................... kernel=rbf, C=10, gamma=0.01, total=  57.6s
[CV] kernel=rbf, C=10, gamma=0.001 ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 25.3min finished


0.84896000000000005

### <font color="darkred">Si algún resultado supera el 90% de accuracy: +1punto (bonus)